# GitHub Group Creation

> Create GitHub group based on the provided group information

In [ ]:
#| default_exp gh_group

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *
import os
import glob
import subprocess

In [ ]:
#| export
def command(cmd: str # command to call in bash
           ) -> (bool, str): # success, output
    "Execute the given command in shell"
    success = False 
    try:
        output = subprocess.check_output(cmd, shell=True, stderr = subprocess.STDOUT)
        success = True 
    except subprocess.CalledProcessError as e:
        output = e.output
    except Exception as e:
        # check_call can raise other exceptions, such as FileNotFoundError
        output = str(e)
    print('> '+cmd)
    print(output.decode())
    return success, output

In [ ]:
#| export
def check_gh_auth() -> bool: # whether you have authenticate
    "Check whether you have gh auth configured"
    msg = command("gh auth status")[1]
    if "You are not logged into any GitHub hosts" in str(msg):
        return False
    return True

In [ ]:
#| export
def creat_repo_cmd(repo_name:str, # the name of the created repository
                   repo_org: str, # the GitHub organization name
                   template_repo:str, # the template repository that the new repo will use
                   private_repo=True  # the visibility of the repository
                   ) -> str: # the command of repo creation
    "Generate the appropriate command for creating GitHub repository"
    cmd = "gh repo create"
    if template_repo is not None:
        cmd = f"{cmd} --template {template_repo}"
    if private_repo:
        cmd = f"{cmd} --private"
    cmd = f"{cmd} --clone {repo_org}/{repo_name}"
    return cmd

In [ ]:
#｜ hide
test_eq(
    'gh repo create --template COGS108/group_template --private --clone COGS108/Group01_SP23',
    creat_repo_cmd("Group01_SP23", "COGS108", "COGS108/group_template")
)
test_eq(
    'gh repo create --private --clone COGS108/Group01_SP23',
    creat_repo_cmd("Group01_SP23", "COGS108", None)
)
test_eq(
    'gh repo create --clone COGS108/Group01_SP23',
    creat_repo_cmd("Group01_SP23", "COGS108", None, private_repo=False)
)

In [ ]:
#| export
def manage_access_cmd(repo_name:str, # the name of the created repository
                      repo_org: str, # the GitHub organization name that the repo belongs to
                      collaorator_id: str, # the GitHub id of the collaborator, or team name if `add_team=True`
                      permission="push",  # the permission to that collaborator
                      add_team=False      # add access to github org's team
                     ) -> str: # the command of repo creation
    "GitHub CLI Command for modifying access priviliges of repo"
    if add_team:
        return (f"gh api -X PUT -f permission={permission} --silent "
                f"/orgs/{repo_org}/teams/{collaorator_id}/repos/{repo_org}/{repo_name}")
    cmd = f"gh api -X PUT -f permission={permission} --silent repos/{repo_org}/{repo_name}/{collaorator_id}"
    return cmd


The team must be in the same organization with the repos' parent organization.

In [ ]:
#| hide

test_eq(
    'gh api -X PUT -f permission=push --silent repos/COGS108/Group01_SP23/scott-yj-yang',
    manage_access_cmd("Group01_SP23", "COGS108", "scott-yj-yang")
)

test_eq(
    'gh api -X PUT -f permission=admin --silent repos/COGS108/Group01_SP23/scott-yj-yang',
    manage_access_cmd("Group01_SP23", "COGS108", "scott-yj-yang", permission="admin")
)

test_eq(
    'gh api -X PUT -f permission=admin --silent /orgs/COGS108/teams/Staff_SP23/repos/COGS108/Group01_SP23',
    manage_access_cmd("Group01_SP23", "COGS108", "Staff_SP23", permission="admin", add_team=True)
)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()